## Connect to LabelOps service

Connect to Label Studio instance providing your `API KEY`

In [8]:
import label_studio_sdk
from label_studio_sdk.labelops import LabelOps

API_KEY = 'dfe99b33d3d233b6556ff4520a8457389d2e7299'
LABEL_STUDIO_URL = 'https://app.heartex.com'
PROJECT_ID = 10552

ls = label_studio_sdk.Client(LABEL_STUDIO_URL, API_KEY)

labelops = LabelOps(api_key=API_KEY)

Connect to Label Studio project and upload model-generated predictions:

In [19]:
from transformers import pipeline

project = ls.get_project(PROJECT_ID)

tasks = project.get_tasks()[:10]

# Create predictions based on Huggingface's models https://huggingface.co/docs/transformers/quicktour
classifier = pipeline("sentiment-analysis")
for task in tasks:
    predicted = classifier(task['data']['text'])[0]
    project.create_prediction(
        model_version='distilbert-base-uncased-finetuned-sst-2-english',
        task_id=task['id'],
        result=predicted['label'],
        score=predicted['score'])


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Initialize new LabelOps project based on Label Studio project ID. This operation could take a while, and only one project could be processed in a time

In [21]:
labelops.initialize_project(PROJECT_ID)

'Project 10552 has been successfully initialized!'

## Explore LabelOps

Search for all texts that contain keyword `"great"` and assign `"Positive"` label to them:

In [30]:
lops1 = [{
    "name": "ContainsGreat",
    "label": "Positive",
    "patterns": [{
        "type": "Contains",
        "query": "great"
    }]
}]

labelops.apply(lops1).view_predictions()[:5]

,field,data,task,start,end,label,score,id
0,text,Works great.,6679358,-1,-1,Positive,1.0,2306
1,text,For the price this was a great deal.,6679897,-1,-1,Positive,1.0,5378
2,text,This is a great deal.,6679488,-1,-1,Positive,1.0,2566
3,text,And the sound quality is great.,6679106,-1,-1,Positive,1.0,1802
4,text,It's a great item.,6679363,-1,-1,Positive,1.0,2316


Perform regular expression search with all keywords that indicate bad attitude:

In [29]:
lops2 = [{
    "name": "MatchBadKeywords",
    "label": "Negative",
    "patterns": [{
        "type": "Regex",
        "query": ".*\s(bad|crap|junk|wors[te]|disappoint|sucks).*"
    }]
}]

labelops.apply(lops2).view_predictions()[:5]

,field,data,task,start,end,label,score,id
0,text,Absolutel junk.,6679359,-1,-1,Negative,1.0,2308
1,text,This pair of headphones is the worst that I ha...,6680027,-1,-1,Negative,1.0,5638
2,text,What a piece of junk.. I lose more calls on th...,6680092,-1,-1,Negative,1.0,5768
3,text,the worst phone from samsung...crap..... this ...,6679710,-1,-1,Negative,1.0,5004
4,text,The only thing that disappoint me is the infra...,6680094,-1,-1,Negative,1.0,5772


Perform full text search:

In [28]:
lops3 = [{
    "name": "FullTextLove",
    "label": "Positive",
    "patterns": [{
        "type": "FullText",
        "query": "love this product",
        "threshold": 3
    }]
}]

labelops.apply(lops3).view_predictions()

,field,data,task,start,end,label,score,id
0,text,Love this headset!,6679365,-1,-1,Positive,1.0,2320
1,text,I love this thing!,6679150,-1,-1,Positive,1.0,1890
2,text,I love this device.,6679415,-1,-1,Positive,1.0,2420
3,text,Love This Phone.,6679464,-1,-1,Positive,1.0,2518
4,text,I love this bluetooth!,6679377,-1,-1,Positive,1.0,2344
5,text,Love this product.,6679171,-1,-1,Positive,1.0,1932
6,text,I love this phone!.,6679580,-1,-1,Positive,1.0,2750


Make fuzzy string matching based on Levenshtein distance equal with less than 3 edits:

In [27]:
lops4 = [{
    "name": "CloseToGreatPhone",
    "label": "Positive",
    "patterns": [{
        "type": "TextDistance",
        "query": "great phone",
        "threshold": 3
    }]
}]

labelops.apply(lops4).view_predictions()[:5]

,field,data,task,start,end,label,score,id
0,text,Great phone.,6679992,-1,-1,Positive,1.0,5568
1,text,Great phone!.,6679283,-1,-1,Positive,1.0,2156
2,text,Great phone.,6679743,-1,-1,Positive,1.0,5070
3,text,Great Phone,6679386,0,11,Positive,1.0,752
4,text,Great phone!.,6679381,-1,-1,Positive,1.0,2352


Enable mixed logic: retrieve all strings that match regex pattern AND contains product mentions:

In [26]:
lops5 = [{
    "name": "IncludesProductRecommendation",
    "label": "Positive",
    "patterns": [{
        "type": "Regex",
        "query": ".*I\swould\srecommend.*"
    }, {
        "type": "NamedEntity",
        "query": "PRODUCT",
        "context": True
    }]
}]

labelops.apply(lops5).view_predictions()

,field,data,task,start,end,label,score,id
0,text,"Overall, I would recommend this phone over the...",6679702,-1,-1,Positive,1.0,4988
1,text,I would recommend purchasing the Jabra JX-10 s...,6679540,-1,-1,Positive,1.0,2670


Search for specific sentiment:

In [25]:
lops6 = [{
    "name": "NegativeSentimentDetected",
    "label": "Negative",
    "patterns": [{
        "type": "Sentiment",
        "query": "negative"
    }]
}]

labelops.apply(lops6).view_predictions()[:5]

,data,field,task,start,end,label,score,id
0,!I definitly recommend!!,text,6679229,-1,-1,Negative,1.0,2048
1,Buyer--Be Very Careful!!!!!.,text,6679230,-1,-1,Negative,1.0,2050
2,The majority of the Logitech earbud headsets f...,text,6679231,-1,-1,Negative,1.0,2052
3,Then I had to continue pairing it periodically...,text,6679770,-1,-1,Negative,1.0,5124
4,Can't upload ringtones from a third party.,text,6679771,-1,-1,Negative,1.0,5126


Create LabelOps based on previously uploaded model predictions:

In [31]:
lops78 = [{
    "name": "PositivePredictionsFromTransformers",
    "label": "Positive",
    "patterns": [{
        "type": "Prediction",
        "query": {
            "model_version": "distilbert-base-uncased-finetuned-sst-2-english",
            "from_name": "label",
            "label": "POSITIVE",
            "score": 0.95
        }
    }]
}, {
    "name": "NegativePredictionsFromTransformers",
    "label": "Negative",
    "patterns": [{
        "type": "Prediction",
        "query": {
            "model_version": "distilbert-base-uncased-finetuned-sst-2-english",
            "from_name": "label",
            "label": "NEGATIVE",
            "score": 0.95
        }
    }]
}]
labelops.apply(lops78).view_predictions()[:5]

,field,data,task,start,end,label,score,id
0,text,I have to jiggle the plug to get it to line up...,6679101,-1,-1,Negative,1.0,1792
1,text,If you have several dozen or several hundred c...,6679102,-1,-1,Negative,1.0,1794
2,text,If you are Razr owner...you must have this!,6679103,-1,-1,Positive,1.0,1796
3,text,"Needless to say, I wasted my money.",6679104,-1,-1,Negative,1.0,1798
4,text,What a waste of money and time!.,6679105,-1,-1,Negative,1.0,1800


## Label tasks

Now combine everything and check LabelOps statistics:

In [33]:
lops_all = lops1 + lops2 + lops3 + lops4 + lops5 + lops6 + lops78
result = labelops.apply(lops_all)
result.view_stats()

,name,coverage,conflict,accuracy
0,ContainsGreat,0.119349,0.001808,0.0
1,MatchBadKeywords,0.066908,0.000000,0.0
2,FullTextLove,0.012658,0.000000,0.0
3,CloseToGreatPhone,0.014467,0.000000,0.0
4,IncludesProductRecommendation,0.003617,0.000000,0.0
5,NegativeSentimentDetected,0.835443,0.003617,0.0
6,PositivePredictionsFromTransformers,0.007233,0.001808,0.0
7,NegativePredictionsFromTransformers,0.010850,0.000000,0.0


Finally create predictions in Label Studio, applying version control:

In [34]:
labelops.commit(result, model_version="LabelOps_Test1")

{'predictions_count': 551}